In [5]:
from scipy import stats
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [6]:
df = pd.read_excel('\ГИГБРЕЙНС\AB тестирование\DZ8\AB_Test_Results.xlsx')
df.shape



(10000, 3)

In [7]:
df = df.groupby(['USER_ID', 'VARIANT_NAME']).agg({'REVENUE':'sum'})
df.reset_index(inplace=True)
df.drop_duplicates('USER_ID', keep=False, inplace=True)
df.shape

(4783, 3)

In [8]:
df.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4783 entries, 0 to 7862
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       4783 non-null   int64  
 1   VARIANT_NAME  4783 non-null   object 
 2   REVENUE       4783 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 149.5+ KB


In [11]:
double = df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': pd.Series.nunique}).query('VARIANT_NAME > 1')


In [12]:
df = df[~df['USER_ID'].isin(double['USER_ID'])].sort_values('USER_ID').reset_index(drop=True)
df

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
1,4,variant,0.0
2,5,variant,0.0
3,6,variant,0.0
4,9,variant,0.0
...,...,...,...
4778,9990,variant,0.0
4779,9992,control,0.0
4780,9993,control,0.0
4781,9995,variant,0.0


In [13]:
df = df.drop_duplicates().reset_index(drop=True)
df
     

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.0
1,4,variant,0.0
2,5,variant,0.0
3,6,variant,0.0
4,9,variant,0.0
...,...,...,...
4778,9990,variant,0.0
4779,9992,control,0.0
4780,9993,control,0.0
4781,9995,variant,0.0


In [14]:
df['REVENUE'].describe()

count    4783.000000
mean        0.135873
std         3.011392
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       196.010000
Name: REVENUE, dtype: float64

In [15]:
df['VARIANT_NAME'].value_counts()

VARIANT_NAME
variant    2393
control    2390
Name: count, dtype: int64

In [16]:
df['conversion'] = df['REVENUE'].apply(lambda x: int(x>0))
df.pivot_table(index='VARIANT_NAME',
               columns='conversion',
               values='USER_ID',
               aggfunc='count',
               margins=True)
     

conversion,0,1,All
VARIANT_NAME,,,
control,2336,54,2390
variant,2351,42,2393
All,4687,96,4783


In [17]:
A = df.query('VARIANT_NAME == "control"')
B = df.query('VARIANT_NAME == "variant"')
n1, n2 = A.shape[0], B.shape[0]
m1, m2 = A[A['conversion']==1].shape[0], B[B['conversion']==1].shape[0]

In [18]:
p1, p2 = m1/n1, m2/n2
p1, p2

(0.022594142259414227, 0.017551190973673213)

In [19]:
alpha = 0.05
pvalue = stats.shapiro(df['REVENUE'])[1]
print('Distribution is ' + ['','not'][pvalue < alpha] + ' normal')

Distribution is not normal


In [20]:
stats.mannwhitneyu(x=A['REVENUE'].values, y=B['REVENUE'].values)

MannwhitneyuResult(statistic=2874161.0, pvalue=0.2104875008271777)

In [21]:
M1, M2 = A['REVENUE'].mean(), B['REVENUE'].mean()
D1, D2 = A['REVENUE'].var(), B['REVENUE'].var()

In [22]:
effect_size = (M1-M2)/np.sqrt(((n1-1)*D1 + (n2-1)*D2)/(n1+n2-2))
effect_size

0.04050088556526385

In [24]:
from statsmodels.stats.power import TTestIndPower

In [25]:
analysis = TTestIndPower()
analysis.solve_power(effect_size=effect_size, alpha=alpha, power=None, nobs1=n1, ratio=n2/n1)

0.2882177037329569

Из-за недостаточного объема выборки, получена статистически незначительную разницу.

Рекомендации: 1.Проверить систему сплитования трафика. 2.Проверить длительность теста, учитывая окно конверсии. 3.Рассчитать требуемый размер выборки и избегать преждевременной остановки теста.

Итог: Не вводить изменения, на которых происходило тестирование группы B. Рекомендуется уточнить или изменить дизай для нового запуска теста.